In [26]:
import numpy as np
import pandas as pd
import pickleshare
import sklearn
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report
import importlib
import climbing_ticks_helper as helper
importlib.reload(helper)
pd.set_option('display.expand_frame_repr', False) # display full data in terminal
%matplotlib inline


In [27]:

df = pd.read_csv('/app/model_ready_ticks.csv')

# first take at a Random Forest model:
X = df.drop(columns=['Attempts', 'Lead Style'])
y = df[['Attempts', 'Lead Style']]
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)

# compute accuracy, recall, precision, f1 score for multi class predictions
def print_classification_report():
    for resp in y_test.columns:
        print(f"Classification report for response variable {resp}:")
        # don't print macro avg or weighted avg
        cr = classification_report(y_test[resp], y_pred_df[resp], zero_division=0, output_dict=True)
        # for Lead Style, display Onsight/Flash, Redpoint and "Fell/Hung" instead of 0, 1, 2
        if resp == 'Lead Style':
            cr['Onsight/Flash'] = cr.pop('0')
            cr['Redpoint'] = cr.pop('1')
            cr['Fell/Hung'] = cr.pop('2')
            cr['accuracy'] = cr.pop('accuracy')
        cr.pop('macro avg', None)
        cr.pop('weighted avg', None)
        prettytable = pd.DataFrame(cr).T
        print(prettytable.round(2))
        print("\n");

print_classification_report()


Classification report for response variable Attempts:
          precision  recall  f1-score  support
1              0.91    0.97      0.94   194.00
2              0.14    0.06      0.08    17.00
3              0.00    0.00      0.00     1.00
4              0.00    0.00      0.00     1.00
accuracy       0.89    0.89      0.89     0.89


Classification report for response variable Lead Style:
               precision  recall  f1-score  support
Onsight/Flash       0.86    0.92      0.89   158.00
Redpoint            0.24    0.17      0.20    24.00
Fell/Hung           0.44    0.39      0.41    31.00
accuracy            0.76    0.76      0.76     0.76




In [30]:
df_pretty = pd.read_csv('/app/grouped_ticks.csv')
df_pretty = df_pretty[df_pretty['RouteID'].isin(X_test['RouteID'])].set_index('RouteID').loc[X_test['RouteID']].reset_index()
df_combined = helper.combine_predictions_with_data(df_pretty, y_pred_df)

# print all rows that got Lead Style wrong
wrong_preds = df_combined[df_combined['Lead Style'] != df_combined['Predicted Lead Style']]
right_preds = df_combined
print(f'We got {len(wrong_preds)} predictions wrong out of {y_pred_df.shape[0]} total predictions.')

We got 51 predictions wrong out of 213 total predictions.


In [32]:
# probability predictions:
proba_predictions = rf.predict_proba(X_test)
combined_df = helper.combine_proba_predictions_with_data(df_readable, proba_predictions)
combined_df.head(10)

,Route,RouteID,Date,Route Type,Alpine,Safety,Avg Stars,Pitches,Rating,OS/F pred,RP pred,F/H pred,Lead Style,1 Attempt pred,2 Attempts pred,3+ Attempts pred,Attempts
0,Bonnie and Clyde,109585903,2023-08-13,Sport,0.0,G,3.4,1,5.12b,0.36,0.57,0.07,Fell/Hung,0.54,0.19,0.27,2
1,Old Bushmills,105718084,2024-04-17,Trad,0.0,G,3.3,1,5.10+,0.99,0.00,0.01,Fell/Hung,1.00,0.00,0.00,1
2,Elastic Man,108259493,2024-09-10,Sport,0.0,G,3.4,1,5.11c,0.69,0.13,0.18,Onsight/Flash,0.89,0.10,0.01,1
3,Meltdown,110629043,2021-07-05,Trad,0.0,G,2.9,5,5.11b,0.78,0.02,0.20,Onsight/Flash,1.00,0.00,0.00,1
4,Ernest Stemmingway,114028771,2019-08-16,Sport,0.0,G,3.0,1,5.11a,0.05,0.36,0.59,Onsight/Flash,0.63,0.09,0.28,1
5,Swedish Fish,108043545,2023-09-28,Sport,0.0,G,3.9,1,5.12a,0.97,0.03,0.00,Redpoint,1.00,0.00,0.00,2
6,Picnic Blank It,112836233,2020-10-10,Sport,0.0,G,2.6,1,5.10c,0.94,0.05,0.01,Onsight/Flash,0.96,0.04,0.00,1
7,Home Wrecker,107233036,2021-07-03,Sport,0.0,G,1.8,1,5.9,0.90,0.06,0.04,Onsight/Flash,0.95,0.04,0.01,1
8,Soul Finger,106605255,2019-07-04,Sport,0.0,G,2.7,1,5.11a,1.00,0.00,0.00,Onsight/Flash,1.00,0.00,0.00,1
9,The Chaser,106310756,2020-11-08,Sport,0.0,G,2.4,1,5.11-,0.04,0.17,0.79,Onsight/Flash,0.19,0.72,0.09,1


In [29]:
# TODO:
# * categorize predictions that are close to the correct answer. EG: predicting an Onsight/Flash when the actual is a Redpoint with only 2 attempts isn't that bad. but predicting an Onsight/Flash when the actual is a Fell/Hung with 5 attempts is bad.
# * categorize predictions that though I woudl do better, vs predictions that thought I would do worse


Stored 'df_pretty' (DataFrame)
